In [1]:
#%% Imports
import torch
import einops

from transformer_lens import HookedTransformer

from typing import List, Callable
from torch import Tensor
from jaxtyping import Float, Int, Bool

import plotly.express as px

torch.set_grad_enabled(False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
#%% Setup model & load data
model = HookedTransformer.from_pretrained('gelu-4l')
model.cfg.use_attn_result = True
model.to(device);

Loaded pretrained model gelu-4l into HookedTransformer
Moving model to device:  cuda


In [15]:
print("W_in  shape:", model.W_in.shape)
print("W_out shape:", model.W_out.shape)
print("b_in  shape:", model.b_in.shape)
print("b_out shape:", model.b_out.shape)

W_in  shape: torch.Size([4, 512, 2048])
W_out shape: torch.Size([4, 2048, 512])
b_in  shape: torch.Size([4, 2048])
b_out shape: torch.Size([4, 512])


In [17]:
W_in = einops.rearrange(model.W_in, 'layer d_model neuron -> layer neuron d_model')
W_out = model.W_out  # (layer, neuron, d_model)

In [18]:
einops.einsum()

torch.Size([4, 2048, 512])

In [21]:
torch.arange(3).float()

tensor([0., 1., 2.])